In [4]:
import tensorflow as tf
import os
import numpy as np

In [6]:
tf.compat.v1.disable_eager_execution()

In [7]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [8]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import vgg19
import tensorflow.keras.backend as K

In [10]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Defining the initial variables

In [34]:
target_image_path = '/home/ahmed/Downloads/ahmed2.jpg'
style_reference_image_path = '/home/ahmed/Downloads/great_wave.jpg'

width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)



# Auxiliary Functions

In [35]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    
    return img

def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    
    return x

# Loading the pretrained VGG19 Network and applying it to the three images

In [36]:
K.set_image_data_format('channels_last')

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
#placeholder that will contain the generated image:
combination_image = K.placeholder((1, img_height, img_width, 3))

#Combining the three images in a single batch:
input_tensor = K.concatenate([
                target_image,
                style_reference_image,
                combination_image], axis=0)

#Builds the VGG19 Network with the batch of 3 images as input:
model= vgg19.VGG19(input_tensor=input_tensor,
                  weights='imagenet',
                  include_top=False)

print('Model Loaded.')

Model Loaded.


# Let's Define the Content  and the style loss

In [37]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

#first let's compute the GRAM Matrix:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    
    channels = 3
    size = img_height * img_width
    
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [38]:
#The Total Variation loss:
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - 
        x[:, 1:, :img_width - 1, :])
    
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - 
        x[:, :img_height - 1, 1:, :])
    
    return K.sum(K.pow(a + b, 1.25))

In [49]:
#Defining the final loss that you'll minimize:

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
#layer used for content loss:
content_layer = 'block5_conv2'
#list of layers used for style loss:
style_layers = ['block1_conv1',
               'block2_conv1',
               'block3_conv1',
               'block4_conv1',
               'block5_conv1']
total_variation_weight = 1e-6
style_weight = 4
content_weight = 1

loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_features,
                                     combination_features)

for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(style_layers)) * sl
    
loss = loss + total_variation_weight * total_variation_loss(combination_image)

# Setting up the gradient-descent process

In [50]:
grads = K.gradients(loss, combination_image)[0]

fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        
        return grad_values
    
evaluator = Evaluator()

# Style-transfer Loop

In [51]:
#importing the necessary libraries
from scipy.optimize import fmin_l_bfgs_b
import imageio
import time

In [52]:
result_prefix = 'my_result'
iterations = 50

x = preprocess_image(target_image_path)
x = x.flatten()

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss,
                                    x,
                                    fprime=evaluator.grads,
                                    maxfun=20)
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imageio.imwrite(fname, img)
    print('Image saved as', fname)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 11636763000.0
Image saved as my_result_at_iteration_0.png
Iteration 0 completed in 21s
Start of iteration 1
Current loss value: 5203428000.0
Image saved as my_result_at_iteration_1.png
Iteration 1 completed in 21s
Start of iteration 2
Current loss value: 3510985200.0
Image saved as my_result_at_iteration_2.png
Iteration 2 completed in 21s
Start of iteration 3
Current loss value: 2804970200.0
Image saved as my_result_at_iteration_3.png
Iteration 3 completed in 21s
Start of iteration 4
Current loss value: 2378099500.0
Image saved as my_result_at_iteration_4.png
Iteration 4 completed in 21s
Start of iteration 5
Current loss value: 2146720600.0
Image saved as my_result_at_iteration_5.png
Iteration 5 completed in 21s
Start of iteration 6
Current loss value: 1933855000.0
Image saved as my_result_at_iteration_6.png
Iteration 6 completed in 21s
Start of iteration 7
Current loss value: 1801688200.0
Image saved as my_result_at_iteration_7.png
Iteration 7 